This notebook shows how a small issue with calculating the solution of a system of differential equations leads to difficulties with solving underdamped systems in sympy.  Fortunately, there is a simple solution!

Ref: https://github.com/sympy/sympy/issues/22715 


In [1]:
import sympy as sym
sym.init_printing()

In [2]:
sym.var('A, b, c0, c1');
sym.var('t', real=True);

Credit for the following process goes to Oscar Benjamin (https://github.com/oscarbenjamin) on the GitHub thread above.  All I did was typeset it.

For certain types of differential systems, sympy calculates the solution to
$$\frac{dx}{dt}=Ax+b$$ using the Jordan normal form of A, specifically finding $P$ and $J_m$ such that:
$$A=PJ_mP^{-1}$$
where $J_m$ is the Jordan normal form of $A$.  

Given this, if we call $J(t)=e^{J_mt}$:
$$
\begin{align*}
e^{At}&=P\,J(t)\,P^{-1}\\
e^{-At}&=\left(P\,J(t)\,P^{-1}\right)^{-1}=P\,J^{-1}(t)\,P^{-1}
\end{align*}$$  

With the terms above, the solution is found as follows:
$$
\begin{align*}
\frac{dx}{dt}&=Ax+b\\
e^{-At}\frac{dx}{dt}&=e^{-At}Ax+e^{-At}b\\
e^{-At}\frac{dx}{dt}-e^{-At}Ax&=e^{-At}b\\
\frac{d}{dt}\left(e^{-At}x\right)&=e^{-At}b\\
e^{-At}x&=\int{e^{-At}b\,dt}+c\\
x&=e^{At}\left(\int{e^{-At}b\,dt}+c\right)\\
x&=P\,J(t)\,P^{-1}\left(\int{P\,J^{-1}(t)\,P^{-1}b\,dt}+c\right)\\
x&=P\,J(t)\left(\int{J^{-1}(t)\,P^{-1}b\,dt}+c'\right)
\end{align*}$$
where in the last line, the unknown constant $c'=P^{-1}c$.

The complication arises for some underdamped systems where sympy ends up with trigonometric functions in the denominator of $J(t)^{-1}$.  This can be resolved by noting that $J(t)^{-1}$ = $J(-t)$.  The following will show where the problem arises and how to solve it for a system of differential equations modeling an underdamped system with eigenvalues of $-1\pm j2$:
$$
\begin{align*}
\frac{dx_1(t)}{dt}&=-x_2(t)+10\\
\frac{dx_2(t)}{dt}&=5x_1(t)-2x_2(t)+3\\
\end{align*}
$$

The solutions should be in the form $e^{-t}\left(\alpha\cos(2t)+\beta\sin(2t)\right)+\gamma$

In [3]:
A = sym.Matrix([[0, -1], [5, -2]])
b = sym.Matrix([[10], [3]])
cv = sym.Matrix([[c0], [c1]])
A, b, cv

⎛⎡0  -1⎤  ⎡10⎤  ⎡c₀⎤⎞
⎜⎢     ⎥, ⎢  ⎥, ⎢  ⎥⎟
⎝⎣5  -2⎦  ⎣3 ⎦  ⎣c₁⎦⎠

In [4]:
P, J = sym.solvers.ode.systems.matrix_exp_jordan_form(A, t)
P, J

⎛             ⎡ -t             -t         ⎤⎞
⎜⎡1/5  -2/5⎤  ⎢ℯ  ⋅cos(2⋅t)  -ℯ  ⋅sin(2⋅t)⎥⎟
⎜⎢         ⎥, ⎢                           ⎥⎟
⎜⎣ 1    0  ⎦  ⎢ -t            -t          ⎥⎟
⎝             ⎣ℯ  ⋅sin(2⋅t)  ℯ  ⋅cos(2⋅t) ⎦⎠

For some underdamped systems, $J^{-1}(t)$ has a problematic form (though it could be simplifed) 

In [5]:
JI1 = J.inv()
JI1

⎡   t    2            t                ⎤
⎢  ℯ ⋅sin (2⋅t)      ℯ       t         ⎥
⎢- ──────────── + ────────  ℯ ⋅sin(2⋅t)⎥
⎢    cos(2⋅t)     cos(2⋅t)             ⎥
⎢                                      ⎥
⎢        t                   t         ⎥
⎣      -ℯ ⋅sin(2⋅t)         ℯ ⋅cos(2⋅t)⎦

In [6]:
sym.simplify(JI1)

⎡ t             t         ⎤
⎢ℯ ⋅cos(2⋅t)   ℯ ⋅sin(2⋅t)⎥
⎢                         ⎥
⎢  t            t         ⎥
⎣-ℯ ⋅sin(2⋅t)  ℯ ⋅cos(2⋅t)⎦

Conveniently, $J^{-1}(t)$ is actually $J(-t)$

In [7]:
JI2 = J.subs(t, -t)
JI2

⎡ t             t         ⎤
⎢ℯ ⋅cos(2⋅t)   ℯ ⋅sin(2⋅t)⎥
⎢                         ⎥
⎢  t            t         ⎥
⎣-ℯ ⋅sin(2⋅t)  ℯ ⋅cos(2⋅t)⎦

In [8]:
sym.simplify(JI1-JI2)

⎡0  0⎤
⎢    ⎥
⎣0  0⎦

This is where things went wrong - note the unevaluated integrals in the answer!  ALso note that the code below is not exactly what is in sympy but acurately represents what happens.

In [9]:
JI1 * P.inv() * b + cv

⎡        t    2            t                 t  ⎤
⎢     3⋅ℯ ⋅sin (2⋅t)   47⋅ℯ ⋅sin(2⋅t)     3⋅ℯ   ⎥
⎢c₀ - ────────────── - ────────────── + ────────⎥
⎢        cos(2⋅t)            2          cos(2⋅t)⎥
⎢                                               ⎥
⎢                               t               ⎥
⎢              t            47⋅ℯ ⋅cos(2⋅t)      ⎥
⎢      c₁ - 3⋅ℯ ⋅sin(2⋅t) - ──────────────      ⎥
⎣                                 2             ⎦

In [10]:
ans1 = P * J * (sym.Integral(JI1 * P.inv() * b, t).doit() + P.inv() * cv)
ans1

⎡⎛                           ⌠               ⌠                  ⎞             
⎢⎜     ⌠                     ⎮      t        ⎮    t    2        ⎟             
⎢⎜     ⎮     t               ⎮  -6⋅ℯ         ⎮ 6⋅ℯ ⋅sin (2⋅t)   ⎟             
⎢⎜     ⎮ 47⋅ℯ ⋅sin(2⋅t) dt + ⎮ ──────── dt + ⎮ ────────────── dt⎟             
⎢⎜     ⌡                     ⎮ cos(2⋅t)      ⎮    cos(2⋅t)      ⎟ ⎛     -t    
⎢⎜                           ⌡               ⌡                  ⎟ ⎜  2⋅ℯ  ⋅sin
⎢⎜c₁ - ─────────────────────────────────────────────────────────⎟⋅⎜- ─────────
⎢⎝                                 2                            ⎠ ⎝        5  
⎢                                                                             
⎢                     ⎛                           ⌠               ⌠           
⎢                     ⎜     ⌠                     ⎮      t        ⎮    t    2 
⎢                     ⎜     ⎮     t               ⎮  -6⋅ℯ         ⎮ 6⋅ℯ ⋅sin (
⎢                     ⎜     ⎮ 47⋅ℯ ⋅sin(2⋅t) dt + ⎮ 

This is how it gets fixed using $J(-t)$ instead of $J^{-1}(t)$

In [11]:
JI2 * P.inv() * b + cv

⎡         t                         ⎤
⎢     47⋅ℯ ⋅sin(2⋅t)      t         ⎥
⎢c₀ - ────────────── + 3⋅ℯ ⋅cos(2⋅t)⎥
⎢           2                       ⎥
⎢                                   ⎥
⎢                         t         ⎥
⎢        t            47⋅ℯ ⋅cos(2⋅t)⎥
⎢c₁ - 3⋅ℯ ⋅sin(2⋅t) - ──────────────⎥
⎣                           2       ⎦

In [12]:
ans2 = P * J * (sym.Integral(JI2 * P.inv() * b, t).doit() +  P.inv() * cv)
ans2

⎡⎛     -t             -t         ⎞ ⎛        t                          ⎞   ⎛  
⎢⎜  2⋅ℯ  ⋅sin(2⋅t)   ℯ  ⋅cos(2⋅t)⎟ ⎜     7⋅ℯ ⋅sin(2⋅t)       t         ⎟   ⎜  
⎢⎜- ────────────── + ────────────⎟⋅⎜c₁ - ───────────── + 10⋅ℯ ⋅cos(2⋅t)⎟ + ⎜- 
⎢⎝        5               5      ⎠ ⎝           2                       ⎠   ⎝  
⎢                                                                             
⎢                     ⎛        t                          ⎞                ⎛  
⎢                     ⎜     7⋅ℯ ⋅sin(2⋅t)       t         ⎟  -t            ⎜  
⎢                     ⎜c₁ - ───────────── + 10⋅ℯ ⋅cos(2⋅t)⎟⋅ℯ  ⋅cos(2⋅t) - ⎜- 
⎣                     ⎝           2                       ⎠                ⎝  

 -t               -t         ⎞ ⎛                                  t         ⎞⎤
ℯ  ⋅sin(2⋅t)   2⋅ℯ  ⋅cos(2⋅t)⎟ ⎜  5⋅c₀   c₁       t            7⋅ℯ ⋅cos(2⋅t)⎟⎥
──────────── - ──────────────⎟⋅⎜- ──── + ── - 10⋅ℯ ⋅sin(2⋅t) - ─────────────⎟⎥
     5               5       ⎠ ⎝   2     2         

In [13]:
sym.trigsimp(ans2)

⎡     -t                                  -t               ⎤
⎢ c₀⋅ℯ  ⋅sin(2⋅t)       -t            c₁⋅ℯ  ⋅sin(2⋅t)   17 ⎥
⎢ ─────────────── + c₀⋅ℯ  ⋅cos(2⋅t) - ─────────────── + ── ⎥
⎢        2                                   2          5  ⎥
⎢                                                          ⎥
⎢      -t                -t                                ⎥
⎢5⋅c₀⋅ℯ  ⋅sin(2⋅t)   c₁⋅ℯ  ⋅sin(2⋅t)       -t              ⎥
⎢───────────────── - ─────────────── + c₁⋅ℯ  ⋅cos(2⋅t) + 10⎥
⎣        2                  2                              ⎦